<a href="https://colab.research.google.com/github/apriandito/nowcasting/blob/main/4_Walk_Forward_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings("ignore")

import requests
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet, LinearRegression, Ridge, Lasso, BayesianRidge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly.graph_objects as go
import matplotlib.pyplot as plt

Get API Key: http://alphavantage.co/

In [2]:
# Ambil data dari API
API_KEY = '8V7F020H9WXQRHLQ'
url = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=USD&to_symbol=IDR&apikey={API_KEY}&outputsize=full"
response = requests.get(url)
data = response.json()
timeseries = data.get("Time Series FX (Daily)", {})
timeseries

{'2025-05-01': {'1. open': '16674.00000',
  '2. high': '16674.00000',
  '3. low': '16594.00000',
  '4. close': '16597.00000'},
 '2025-04-30': {'1. open': '16782.00000',
  '2. high': '16782.00000',
  '3. low': '16589.00000',
  '4. close': '16594.00000'},
 '2025-04-29': {'1. open': '16857.00000',
  '2. high': '16857.00000',
  '3. low': '16753.00000',
  '4. close': '16754.00000'},
 '2025-04-28': {'1. open': '16824.00000',
  '2. high': '16863.00000',
  '3. low': '16819.00000',
  '4. close': '16849.00000'},
 '2025-04-25': {'1. open': '16799.00000',
  '2. high': '16879.00000',
  '3. low': '16799.00000',
  '4. close': '16824.00000'},
 '2025-04-24': {'1. open': '16875.00000',
  '2. high': '16884.00000',
  '3. low': '16849.00000',
  '4. close': '16864.00000'},
 '2025-04-23': {'1. open': '16857.00000',
  '2. high': '16884.00000',
  '3. low': '16844.00000',
  '4. close': '16859.00000'},
 '2025-04-22': {'1. open': '16803.00000',
  '2. high': '16864.00000',
  '3. low': '16799.00000',
  '4. close': 

In [3]:
# Konversi ke DataFrame
df = pd.DataFrame.from_dict(timeseries, orient='index')
df.columns = ['Open', 'High', 'Low', 'Close']
df.index = pd.to_datetime(df.index)
df = df.sort_index()
df = df.astype(float)
df

,Open,High,Low,Close
2014-11-07,12120.0,12170.0,12115.0,12135.0
2014-11-10,12170.0,12170.0,12135.0,12150.0
2014-11-11,12200.0,12200.0,12200.0,12200.0
2014-11-12,12200.0,12218.0,12142.0,12197.0
2014-11-13,12197.0,12210.0,12142.0,12200.0
...,...,...,...,...
2025-04-25,16799.0,16879.0,16799.0,16824.0
2025-04-28,16824.0,16863.0,16819.0,16849.0
2025-04-29,16857.0,16857.0,16753.0,16754.0
2025-04-30,16782.0,16782.0,16589.0,16594.0


In [4]:
# Buat salinan dan fitur tambahan
df_feat = df.copy()

# Fitur hari ini (Open hari ini) — masih diperbolehkan
df_feat['Open_today'] = df_feat['Open']

# Fitur hari sebelumnya (lag 1)
df_feat['Close_t-1'] = df_feat['Close'].shift(1)
df_feat['Open_t-1'] = df_feat['Open'].shift(1)
df_feat['High_t-1'] = df_feat['High'].shift(1)
df_feat['Low_t-1'] = df_feat['Low'].shift(1)

# Moving Averages dari harga Close historis (hindari leakage)
df_feat['MA_3'] = df_feat['Close'].shift(1).rolling(window=3).mean()
df_feat['MA_5'] = df_feat['Close'].shift(1).rolling(window=5).mean()

# Rolling Standard Deviation (historis)
df_feat['STD_3'] = df_feat['Close'].shift(1).rolling(window=3).std()
df_feat['STD_5'] = df_feat['Close'].shift(1).rolling(window=5).std()

# Rate of Change (ROC) historis
df_feat['ROC_1'] = df_feat['Close'].shift(1).pct_change(periods=1)
df_feat['ROC_3'] = df_feat['Close'].shift(1).pct_change(periods=3)

# Momentum historis
df_feat['Momentum_1'] = df_feat['Close'].shift(1) - df_feat['Close'].shift(2)
df_feat['Momentum_3'] = df_feat['Close'].shift(1) - df_feat['Close'].shift(4)

# Hapus baris dengan missing values
df_feat.dropna(inplace=True)

In [11]:
# Pisahkan baris terakhir untuk prediksi real-time
latest_row = df_feat.iloc[[-1]]
df_feat = df_feat.iloc[:-1]

# Siapkan fitur dan target (tambahkan fitur baru)
X = df_feat[[
    'Open_today', 'Close_t-1', 'Open_t-1', 'High_t-1', 'Low_t-1',
    'MA_3', 'MA_5', 'STD_3', 'STD_5', 'ROC_1', 'ROC_3', 'Momentum_1', 'Momentum_3'
]]
X

,Open_today,Close_t-1,Open_t-1,High_t-1,Low_t-1,MA_3,MA_5,STD_3,STD_5,ROC_1,ROC_3,Momentum_1,Momentum_3
2014-11-14,12200.0,12200.0,12197.0,12210.0,12142.0,12199.000000,12176.4,1.732051,31.421330,0.000246,0.004115,3.0,50.0
2014-11-17,12210.0,12210.0,12200.0,12218.0,12200.0,12202.333333,12191.4,6.806859,23.660093,0.000820,0.000820,10.0,10.0
2014-11-18,12202.0,12202.0,12210.0,12210.0,12185.0,12204.000000,12201.8,5.291503,4.919350,-0.000655,0.000410,-8.0,5.0
2014-11-19,12145.0,12145.0,12202.0,12210.0,12065.0,12185.666667,12190.8,35.444793,26.051871,-0.004671,-0.004508,-57.0,-55.0
2014-11-20,12145.0,12145.0,12145.0,12145.0,12120.0,12164.000000,12180.4,32.908965,32.531523,0.000000,-0.005324,0.0,-65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-23,16857.0,16849.0,16803.0,16864.0,16799.0,16822.333333,16821.0,25.166115,17.888544,0.002976,0.001784,50.0,30.0
2025-04-24,16875.0,16859.0,16857.0,16884.0,16844.0,16835.666667,16829.0,32.145503,24.494897,0.000594,0.002378,10.0,40.0
2025-04-25,16799.0,16864.0,16875.0,16884.0,16849.0,16857.333333,16838.0,7.637626,27.928480,0.000297,0.003869,5.0,65.0
2025-04-28,16824.0,16824.0,16799.0,16879.0,16799.0,16849.000000,16839.0,21.794495,27.156951,-0.002372,-0.001484,-40.0,-25.0


In [12]:
y = df_feat['Close']
y

,Close
2014-11-14,12210.0
2014-11-17,12202.0
2014-11-18,12145.0
2014-11-19,12145.0
2014-11-20,12170.0
...,...
2025-04-23,16859.0
2025-04-24,16864.0
2025-04-25,16824.0
2025-04-28,16849.0


In [6]:
# Daftar model
models = {
    'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5),
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.01),
    'BayesianRidge': BayesianRidge(),
}

# Walk-forward validation
validation_results = []
start_index = 60  # Awal rolling window

for t in range(start_index, len(X)):
    X_train, y_train = X.iloc[:t], y.iloc[:t]
    X_test, y_test_point = X.iloc[t:t+1], y.iloc[t:t+1]

    result = {
        'Date': y_test_point.index[0],
        'Actual': y_test_point.values[0]
    }

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)[0]
        result[f'{name}_Pred'] = y_pred
        result[f'{name}_AbsErr'] = abs(y_pred - y_test_point.values[0])

    validation_results.append(result)

# Simpan hasil ke DataFrame
val_df = pd.DataFrame(validation_results)
val_df.set_index('Date', inplace=True)

# Evaluasi akurasi model
metrics = []
for name in models.keys():
    y_true = val_df['Actual']
    y_pred = val_df[f'{name}_Pred']
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = r2_score(y_true, y_pred)
    metrics.append({
        'Model': name,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2
    })

# Hasil evaluasi
results_df = pd.DataFrame(metrics)
results_df.round(4)

,Model,MAE,MSE,RMSE,MAPE,R2
0,ElasticNet,37.2962,3306.2002,57.4996,0.2575,0.9962
1,LinearRegression,37.4764,3485.6950,59.0398,0.2586,0.9960
2,Ridge,37.3279,3342.5842,57.8151,0.2578,0.9962
3,Lasso,37.2969,3306.2978,57.5004,0.2575,0.9962
4,BayesianRidge,37.0492,3286.4793,57.3278,0.2557,0.9963


In [7]:
# Visualisasi prediksi model terbaik
predictions = {name: val_df[f'{name}_Pred'].values for name in models.keys()}
y_test = val_df['Actual']

# Plotly interactive graph
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=val_df.index,
    y=y_test,
    mode='lines',
    name='Actual',
    line=dict(width=2),
    visible=True
))

for i, (name, y_pred) in enumerate(predictions.items()):
    fig.add_trace(go.Scatter(
        x=val_df.index,
        y=y_pred,
        mode='lines',
        name=f'Predicted - {name}',
        line=dict(width=2),
        visible=(i == 0)
    ))

dropdown_buttons = []
n_models = len(predictions)
for i, name in enumerate(predictions.keys()):
    visibility = [True] + [j == i for j in range(n_models)]
    dropdown_buttons.append(dict(
        label=name,
        method='update',
        args=[
            {'visible': visibility},
            {'title': f'{name} Prediction of USD/IDR Closing Price'}
        ]
    ))

fig.update_layout(
    title='Model Prediction of USD/IDR Closing Price (Walk-Forward)',
    xaxis_title='Date',
    yaxis_title='Closing Price (IDR)',
    legend=dict(x=0.01, y=0.99),
    template='plotly_white',
    width=1000,
    height=500,
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction="down",
        showactive=True,
        x=1.15,
        xanchor="left",
        y=1.15,
        yanchor="top"
    )]
)
fig.show()

In [8]:
# Pilih model terbaik secara manual
best_model_name = 'BayesianRidge'  # ← Ganti sesuai model yang Anda pilih
best_model = models[best_model_name]

In [9]:
X_latest = latest_row[[
    'Open_today', 'Close_t-1', 'Open_t-1', 'High_t-1', 'Low_t-1',
    'MA_3', 'MA_5', 'STD_3', 'STD_5', 'ROC_1', 'ROC_3', 'Momentum_1', 'Momentum_3'
]]
X_latest

,Open_today,Close_t-1,Open_t-1,High_t-1,Low_t-1,MA_3,MA_5,STD_3,STD_5,ROC_1,ROC_3,Momentum_1,Momentum_3
2025-05-01,16674.0,16594.0,16782.0,16782.0,16589.0,16732.333333,16777.0,128.873323,110.657128,-0.00955,-0.013671,-160.0,-230.0


In [10]:
# 🔮 Prediksi hari terakhir (yang tadi dipisahkan)

y_actual = latest_row['Close'].values[0]
y_forecast = best_model.predict(X_latest)[0]
absolute_error = abs(y_forecast - y_actual)

print("\nReal-time Forecast (Simulasi Hari Terakhir):")
print(f"Tanggal         : {latest_row.index[0].date()}")
print(f"Actual Close    : {y_actual:.2f}")
print(f"Predicted Close : {y_forecast:.2f}")
print(f"Absolute Error   : {absolute_error:.2f}")


Real-time Forecast (Simulasi Hari Terakhir):
Tanggal         : 2025-05-01
Actual Close    : 16597.00
Predicted Close : 16608.27
Absolute Error   : 11.27
